In [4]:
###############################################################################################
# PlanetScope Imagery Preprocessing                                                           #
# To automatically perform ToA and DOS correction on PlanetScope Imagery                      #
# image product name is generally composed of the following elements:                         #
# <acquisition date>_<acquisition time>_<satellite id>_<productLevel><bandProduct>.<extension>#
###############################################################################################

In [11]:
#import dependencies and set the working paths 
import rasterio
import shutil
import pandas
import numpy
import math
import glob
import sys
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from osgeo import gdal
from osgeo.gdalconst import *


path = r"C:\Users\envisage\Desktop\LULC Mapping\laguna_planet_q1"
foldername = r"C:\Users\envisage\Desktop\LULC Mapping\Laguna"
gdal_merge_path = r"C:\Users\Rudy\AppData\Local\Programs\Python\Python36\GDAL-2.2.3"

#define xml abbreviations
ps="{http://schemas.planet.com/ps/v1/planet_product_metadata_geocorrected_level}"
eop="{http://earth.esa.int/eop}"
gml="{http://www.opengis.net/gml}"
opt="{http://earth.esa.int/opt}"

In [18]:
coeff = {}
for root, dirs, files in os.walk (path):
    for name in files:
        if name.endswith((".xml")) and not name.startswith(('.')):
            print ('xml file:', name)
        
            data = ET.parse(root + "/" + name)
            tags = data.getroot()
            
            coeff_temp = {name.split('_metadata')[0]:{'rc_b1': '', 'rc_b2': '', 'rc_b3': '', 'rc_b4': ''}}
            
            for member in tags.findall(gml+'resultOf'):
                for submember in member.findall(ps+'EarthObservationResult'):
                    for info in submember.findall(ps+'bandSpecificMetadata'):
                        band = info.find(ps+'bandNumber').text
                        rf = info.find(ps+'reflectanceCoefficient').text
                        
                        coeff_temp[name.split('_metadata')[0]]['rc_b'+band] = rf
            
            coeff.update(coeff_temp)
         
            #os.mkdir(r'C:/Users/Rudy/Desktop/KII/planet_order_125125/20151225_003546_1_0c74/%s'%name.split('_metadata')[0])
            tmpfolderName = foldername+'/%s'%name.split('_metadata')[0]
            print (tmpfolderName)
            
            for i in range (len(coeff)):
                for j in range (len(coeff[name.split('_metadata')[i]])):
                    print ('rc_b'+str(j+1)+': ', coeff[name.split('_metadata')[i]]['rc_b'+str(j+1)])  
                        
            image = root+"/"+name.split('_metadata')[0]+'.tif'
            raster = gdal.Open(image)
        
            if raster is None: 
                print ('Unable to Open Raster Dataset', raster)
        
            for b in range (raster.RasterCount + 1)[1::]:
                with rasterio.open(image) as src:
                    band = src.read(b)
                    band = band*float(coeff[name.split('_metadata')[0]]['rc_b%s'%b])
                    kwargs = src.meta 
                    kwargs.update(dtype=rasterio.float32, count=1)
                    with rasterio.open (tmpfolderName+'_'+str(b)+'.tiff', 'w', **kwargs) as dst:
                        dst.write_band(1, band.astype(rasterio.float32))
            
            outvrt = '/vsimem/stacked.vrt'
            outtif = foldername+'/%s'%name.split('_metadata')[0]+'_TOA.tif'
            
            merge_command_2 = sorted(glob.glob(foldername+"/*.tiff"))
            #print (merge_command_2)
            
            outds = gdal.BuildVRT(outvrt, merge_command_2, separate=True)
            outds = gdal.Translate(outtif, outds)
            
            #remove the genarated tif file 
            for b in range (raster.RasterCount + 1)[1::]:
                file = tmpfolderName+'_'+str(b)+'.tiff'
                if os.path.isfile(file):
                    os.remove(file)
                else:
                    print ("Error: %s file not found"%file)
                    
            del(coeff[name.split('_metadata')[0]])  
            

print ('Done')
        
            

xml file: 20170103_050728_0c76_3B_AnalyticMS_metadata.xml
{'20170103_050728_0c76_3B_AnalyticMS': {'rc_b1': '2.0369393579e-05', 'rc_b2': '2.17661336798e-05', 'rc_b3': '2.42953012781e-05', 'rc_b4': '3.67509029687e-05'}}
C:\Users\envisage\Desktop\LULC Mapping\Laguna/20170103_050728_0c76_3B_AnalyticMS
0
0
rc_b1:  2.0369393579e-05
1
rc_b2:  2.17661336798e-05
2
rc_b3:  2.42953012781e-05
3
rc_b4:  3.67509029687e-05
xml file: 20170103_050729_0c76_3B_AnalyticMS_metadata.xml
{'20170103_050729_0c76_3B_AnalyticMS': {'rc_b1': '2.03620279486e-05', 'rc_b2': '2.17582629843e-05', 'rc_b3': '2.42865160284e-05', 'rc_b4': '3.67376137381e-05'}}
C:\Users\envisage\Desktop\LULC Mapping\Laguna/20170103_050729_0c76_3B_AnalyticMS
0
0
rc_b1:  2.03620279486e-05
1
rc_b2:  2.17582629843e-05
2
rc_b3:  2.42865160284e-05
3
rc_b4:  3.67376137381e-05
xml file: 20170103_050730_0c76_3B_AnalyticMS_metadata.xml
{'20170103_050730_0c76_3B_AnalyticMS': {'rc_b1': '2.03543347076e-05', 'rc_b2': '2.17500422136e-05', 'rc_b3': '2.4277

xml file: 20170225_014144_0e1f_3B_AnalyticMS_metadata.xml
{'20170225_014144_0e1f_3B_AnalyticMS': {'rc_b1': '2.2255636069e-05', 'rc_b2': '2.33041596513e-05', 'rc_b3': '2.61430792149e-05', 'rc_b4': '3.87958320435e-05'}}
C:\Users\envisage\Desktop\LULC Mapping\Laguna/20170225_014144_0e1f_3B_AnalyticMS
0
0
rc_b1:  2.2255636069e-05
1
rc_b2:  2.33041596513e-05
2
rc_b3:  2.61430792149e-05
3
rc_b4:  3.87958320435e-05
xml file: 20170225_014145_0e1f_3B_AnalyticMS_metadata.xml
{'20170225_014145_0e1f_3B_AnalyticMS': {'rc_b1': '2.22456948723e-05', 'rc_b2': '2.32937500978e-05', 'rc_b3': '2.61314015666e-05', 'rc_b4': '3.87785026355e-05'}}
C:\Users\envisage\Desktop\LULC Mapping\Laguna/20170225_014145_0e1f_3B_AnalyticMS
0
0
rc_b1:  2.22456948723e-05
1
rc_b2:  2.32937500978e-05
2
rc_b3:  2.61314015666e-05
3
rc_b4:  3.87785026355e-05
xml file: 20170225_014147_0e1f_3B_AnalyticMS_metadata.xml
{'20170225_014147_0e1f_3B_AnalyticMS': {'rc_b1': '2.22357924971e-05', 'rc_b2': '2.32833811947e-05', 'rc_b3': '2.6119

In [40]:
##########################################################################
#Dark Object Subtraction 

folderpath = r"C:\Users\envisage\Desktop\LULC Mapping\Laguna"
foldername_DOS = r"C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS"

for root, dirs, files in os.walk (folderpath):
    for name in files:
        if name.endswith(("_TOA.tif")):     
            #os.mkdir(r'C:/Users/Rudy/Desktop/KII/planet_order_125125/20151225_003546_1_0c74/%s'%name.split('_metadata')[0])
            tmpfolderName = foldername_DOS +'/%s'%name.split('.tif')[0]
            
            ras = root+"/"+name
            ds = gdal.Open(folderpath+'/%s'%name)
        
            if ds is None:
                print ('Unable to Open Raster Dataset', ds)
            
        
            for b in range (ds.RasterCount + 1)[1::]:
                raster_band = ds.GetRasterBand(b)
                stat = raster_band.GetStatistics(True, True)
                b_min = stat[0]
                b_max = stat[1]
                with rasterio.open(ras) as src:
                    band = src.read(b, masked=True)
                    band = band - float(b_min)
                    kwargs = src.meta 
                    kwargs.update(dtype=rasterio.float32, count=1)
                    with rasterio.open (tmpfolderName+'_'+str(b)+'.tiff', 'w', **kwargs) as dst:
                        dst.write_band(1, band.astype(rasterio.float32))
            
            outvrt = '/vsimem/stacked.vrt'
            outtif = foldername_DOS+'/%s'%name.split('.tif')[0]+'_DOS.tif'
            print (outtif)
            
            merge_command_2 = sorted(glob.glob(foldername_DOS+"/*.tiff"))
            #print (merge_command_2)
            
            outds = gdal.BuildVRT(outvrt, merge_command_2, separate=True)
            outds = gdal.Translate(outtif, outds)
            
            #remove the genarated tif file 
            for b in range (raster.RasterCount + 1)[1::]:
                file = tmpfolderName+'_'+str(b)+'.tiff'
                if os.path.isfile(file):
                    os.remove(file)
                else:
                    print ("Error: %s file not found"%file)
                    
print ('Done')

Tiff file: 20170103_050728_0c76_3B_AnalyticMS_TOA.tif
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050728_0c76_3B_AnalyticMS_TOA
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050728_0c76_3B_AnalyticMS_TOA_DOS.tif
Tiff file: 20170103_050729_0c76_3B_AnalyticMS_TOA.tif
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050729_0c76_3B_AnalyticMS_TOA
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050729_0c76_3B_AnalyticMS_TOA_DOS.tif
Tiff file: 20170103_050730_0c76_3B_AnalyticMS_TOA.tif
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050730_0c76_3B_AnalyticMS_TOA
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050730_0c76_3B_AnalyticMS_TOA_DOS.tif
Tiff file: 20170103_050731_0c76_3B_AnalyticMS_TOA.tif
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050731_0c76_3B_AnalyticMS_TOA
C:\Users\envisage\Desktop\LULC Mapping\Laguna_DOS/20170103_050731_0c76_3B_AnalyticMS_TOA_DOS.tif
Tiff file: 20170103_050732_1_0c76_3B_Ana

In [9]:
##################################################################
#Calculate Indices

for item in os.listdir(path):
    if item.endswith(("_DOS.tif")):
        with rasterio.open(path+'/%s'%item) as src:
            b1 = src.read(1)
        with rasterio.open(path+'/%s'%item) as src:
            b2 = src.read(2)
        with rasterio.open(path+'/%s'%item) as src:
            b3 = src.read(3)
        with rasterio.open(path+'/%s'%item) as src:
            b4 = src.read(4)
            
        numpy.seterr(divide='ignore', invalid='ignore')
        
        ndvi = (b4.astype(float) - b3.astype(float))/(b4 + b3)
        ndwi = (b3.astype(float) - b4.astype(float))/(b3 + b4)
        savi = ((1+0.5)*(b4.astype(float) - b3.astype(float)))/(b4 + b3 + 0.5)
        redi = (b3.astype(float) - b2.astype(float))/(b3 + b2)
        brti = (b3.astype(float) + b2.astype(float) + b1.astype(float))/3
        
        kwargs = src.meta
        kwargs.update(dtype=rasterio.float32, count=1)
        
        with rasterio.open(path+'\%s'%item.split('.tif')[0]+'_ndvi.tif', 'w', **kwargs) as dst:
            dst.write_band(1, ndvi.astype(rasterio.float32))
            
        with rasterio.open(path+'\%s'%item.split('.tif')[0]+'_ndwi.tif', 'w', **kwargs) as dst:
            dst.write_band(1, ndwi.astype(rasterio.float32))
            
        with rasterio.open(path+'\%s'%item.split('.tif')[0]+'_savi.tif', 'w', **kwargs) as dst:
            dst.write_band(1, savi.astype(rasterio.float32))
        
        with rasterio.open(path+'\%s'%item.split('.tif')[0]+'_redi.tif', 'w', **kwargs) as dst:
            dst.write_band(1, redi.astype(rasterio.float32))
        
        with rasterio.open(path+'\%s'%item.split('.tif')[0]+'_brti.tif', 'w', **kwargs) as dst:
            dst.write_band(1, brti.astype(rasterio.float32))
        
        #showImage(ndvi,'NDVI')

print ('Done')

Done
